In [1]:
import numpy as np

import sys
print(sys.getrecursionlimit())
sys.setrecursionlimit(10000)

3000


In [14]:
class Day20():

    def __init__(self, data_link):
        self.data_flat = open(data_link, 'r').read().split('\n')[:-1]
        self.board = np.array([[elem for elem in line] for line in self.data_flat], dtype = '<U8')
        self.start = np.argwhere(self.board == 'S')[0]
        self.end = np.argwhere(self.board == 'E')[0]
        self.directions = [(0, 1), (1, 0), (0, -1), (-1, 0)]

    def __str__(self):
        return '\n'.join([''.join(list(line)) for line in self.board])

    def sum_tuple(self, tuple_a, tuple_b):
        return tuple([tuple_a[index] + tuple_b[index] for index in range(len(tuple_a))]) if len(tuple_a)==len(tuple_b) else False

    def get_neighbours(self, pos):
        if pos[0] in [0, len(self.board)-1] or pos[1] in [0, len(self.board)-1]:
            return []
        return [self.sum_tuple(pos, direction) for direction in self.directions if self.board[self.sum_tuple(pos, direction)] in ['E', '.']]

    def solve(self, pos = None, dist = 0):
        if not pos:
            pos = tuple(self.start)
        if self.board[pos] == 'E':
            self.board[pos] = 0
            return 0
        self.board[pos] = 'x'
        for neigh in self.get_neighbours(pos):
            dist = 1 + self.solve(neigh)
        self.board[pos] = dist
        return dist

    def get_walls(self):
        return [wall for wall in np.argwhere(self.board == '#') if len(self.get_neighbours(wall)) >= 2]

In [15]:
day20_test = Day20('other/day20_test.txt')
print(day20_test)
day20_test.start, day20_test.end

###############
#...#...#.....#
#.#.#.#.#.###.#
#S#...#.#.#...#
#######.#.#.###
#######.#.#...#
#######.#.###.#
###..E#...#...#
###.#######.###
#...###...#...#
#.#####.#.###.#
#.#...#.#.#...#
#.#.#.#.#.#.###
#...#...#...###
###############


(array([3, 1]), array([7, 5]))

In [16]:
day20_test.get_walls()

[array([1, 4]),
 array([1, 8]),
 array([2, 2]),
 array([2, 4]),
 array([2, 6]),
 array([2, 8]),
 array([ 2, 10]),
 array([ 2, 11]),
 array([ 2, 12]),
 array([3, 6]),
 array([3, 8]),
 array([ 3, 10]),
 array([4, 8]),
 array([ 4, 10]),
 array([ 4, 12]),
 array([ 4, 13]),
 array([5, 8]),
 array([ 5, 10]),
 array([6, 8]),
 array([ 6, 11]),
 array([ 6, 12]),
 array([7, 6]),
 array([ 7, 10]),
 array([8, 2]),
 array([8, 4]),
 array([8, 7]),
 array([8, 8]),
 array([8, 9]),
 array([ 8, 12]),
 array([ 8, 13]),
 array([ 9, 10]),
 array([10,  2]),
 array([10,  3]),
 array([10,  8]),
 array([10, 11]),
 array([10, 12]),
 array([11,  2]),
 array([11,  6]),
 array([11,  8]),
 array([11, 10]),
 array([12,  2]),
 array([12,  4]),
 array([12,  6]),
 array([12,  8]),
 array([12, 10]),
 array([12, 12]),
 array([13,  4]),
 array([13,  8])]

In [138]:
day20 = Day20('data/day20.txt')
print(len(np.argwhere(day20.board == '.')))
print(day20.solve())
print(len(np.argwhere(day20.board == '.')))

9379
9380
0
